In [31]:
def get_target(y_data) :
    
    y_values = []
    
    for item in y_data:
        
        if item not in y_values:
            y_values.append(item)
            
    return y_values

In [32]:
def getFrequency(y_data, label) :
    
    length = len(y_data)
    target = get_target(y_data)
    frequency = 0 
    matching = 0
        
    for item in y_data :
            
        if label == item :
                
            matching=matching+1
                
    frequency = matching/length
    
    return frequency           

In [33]:
import math

def getEntropy(y_data) :
    
    entropy = 0
    target = get_target(y_data)
    
    for tg in target :
        
        f = getFrequency(y_data,tg)
        entropy = entropy - f*math.log(f,2)
        print(entropy)
    
    return entropy

In [36]:
import numpy as np

def getProbability(column) :
    
    length = len(column)
    target = get_target(column)
    probability = []
    
    for tg in target :
        matching = 0
        for item in column :
            
            if tg == item :
                matching=matching+1
                
            prob = matching/length
        probability.append(prob)
    
    return probability

In [37]:
from sklearn import datasets
import numpy as np

X_data= np.array ([['Sunny','Hot','High','Weak'],
        ['Sunny','Hot','High','Strong'],
        ['Overcast','Hot','High','Weak'],
        ['Rain','Mild','High','Weak'],
        ['Rain','Cool','Normal','Weak'],
        ['Rain','Cool','Normal','Strong'],
        ['Overcast','Cool','Normal','Strong'],
        ['Sunny','Mild','High','Weak'],
        ['Sunny','Cool','Normal','Weak'],
        ['Rain','Mild','Normal','Weak'],
        ['Sunny','Mild','Normal','Strong'],
        ['Overcast','Mild','High','Strong'],
        ['Overcast','Hot','Normal','Weak'],
        ['Rain','Mild','High','Strong']])

y_data=np.array(['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no'])

#getEntropy(y_data)

#X_data[:, [3]]

#getProbability(X_data[:, [3]])



[0.5714285714285714, 0.42857142857142855]